In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from impyute.imputation.cs import mice
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [30]:
df = pd.read_excel('OUTPUT.xlsx')
# df.head()

In [31]:
df_imputation_ans = df.drop(columns=['ID', 'name', 'smiles'])
df_imputation_ans = df_imputation_ans.dropna() 
selected_features = df_imputation_ans.columns

scaler = MinMaxScaler()
df_imputation_ans = scaler.fit_transform(df_imputation_ans)
df_imputation_ans = pd.DataFrame(df_imputation_ans, columns=selected_features)
# df_imputation_ans.head()

In [32]:
# 產生要補值的格子的索引
np.random.seed(77)  
total_cells = df_imputation_ans.size  # 總格子數
desired_missing_cells = int(0.05 * total_cells)  # 總格子數的5%
missing_indices = np.random.choice(df_imputation_ans.size, desired_missing_cells, replace=False)
missing_indices

array([ 117921,  638018,  857566, ..., 1018944, 1009477,  313104])

In [33]:
# 將這些格子設置為缺失值
df_imputation_test = df_imputation_ans.copy()
for index in missing_indices:
    row_index = index // df_imputation_test.shape[1]
    col_index = index % df_imputation_test.shape[1]
    df_imputation_test.iat[row_index, col_index] = np.nan

In [34]:
df_imputation_test.head()

,output,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,...,P1s,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds
0,0.0,0.00,0.501084,0.040150,0.202605,0.286901,0.375,0.40,0.254237,0.314815,...,0.833552,0.490494,0.020726,0.445607,0.035581,NaN,0.199219,0.102111,0.121147,0.017402
1,0.0,0.00,0.140205,0.602583,0.650355,0.640699,0.000,0.00,0.661017,0.740741,...,0.838236,0.414251,0.021929,0.576181,0.066570,0.250303,0.426480,NaN,0.127120,0.036974
2,0.0,0.00,0.485258,0.050908,0.298172,0.380742,0.250,0.24,0.364407,0.370370,...,0.844029,0.400977,0.013922,0.373372,0.057665,0.176120,0.230711,0.132229,0.134509,0.016856
3,0.0,0.00,0.591785,0.003095,0.226127,0.307697,0.250,0.24,0.288136,0.296296,...,0.843477,0.388692,0.017992,0.455387,0.073349,0.147084,NaN,0.107358,0.133805,0.029978
4,0.0,0.25,0.665038,0.003753,0.149123,0.298533,0.500,0.48,0.228814,0.351852,...,NaN,0.815149,0.011903,0.472714,0.055431,0.099732,0.211259,0.109478,0.057492,0.020323


In [35]:
# 使用KNN補值
ans = df_imputation_ans.values.flat[missing_indices]
for n in range(1, 10):
    knn_imputer = KNNImputer(n_neighbors=n)
    knn_imputed_df = knn_imputer.fit_transform(df_imputation_test)
    knn_rmse = np.sqrt(mean_squared_error(knn_imputed_df.flat[missing_indices], ans))
    print(n,"KNN RMSE:", knn_rmse)

1 KNN RMSE: 0.09086977261538981
2 KNN RMSE: 0.08200146086289702
3 KNN RMSE: 0.07945302915461155
4 KNN RMSE: 0.07886279194618365
5 KNN RMSE: 0.07894965821430767
6 KNN RMSE: 0.0794336029301825
7 KNN RMSE: 0.07988279129923671
8 KNN RMSE: 0.08035248851358352
9 KNN RMSE: 0.08092585982757106


In [22]:
# 使用MICE補值，會跑很久
mice_imputed_df = mice(df_imputation_test.values)
mice_rmse = np.sqrt(mean_squared_error(mice_imputed_df.flat[missing_indices], ans))
print(mice_rmse)

KeyboardInterrupt: 

In [14]:
data = df.drop(columns=['ID', 'name', 'smiles'])
# data.head()

In [15]:
inf_locations = np.isinf(data)
print("無窮大值的位置：")
print(np.where(inf_locations))

無窮大值的位置：
(array([416, 416, 416, 416, 416, 416, 416, 416, 416, 416, 416, 416, 416,
       416, 416, 416, 416, 416, 416, 416, 416, 416, 416, 416, 416, 416,
       416, 416, 416, 416, 416, 416, 416, 416, 416, 416, 416, 416, 416,
       416, 416, 416, 416, 416, 416, 416, 416, 416, 416, 416, 416, 416,
       416, 416, 416, 416]), array([ 66, 120, 184, 185, 186, 187, 188, 189, 190, 191, 247, 248, 249,
       250, 251, 252, 253, 254, 730, 731, 733, 756, 758, 761, 762, 764,
       765, 777, 778, 779, 780, 862, 863, 865, 888, 890, 893, 894, 896,
       897, 909, 910, 911, 912, 920, 921, 923, 927, 928, 929, 930, 931,
       932, 962, 963, 964]))


In [16]:
for column in data.columns:
    max_value = data[column][data[column] != np.inf].max()  # Calculate the maximum value excluding inf
    data[column].replace(np.inf, max_value, inplace=True)  # Replace inf with the maximum value

In [17]:
data.iloc[416, 66]

1247.874914266118

In [19]:
# 使用KNN補值原始資料的NaN
scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(data)

# 使用KNN補值
knn_imputer = KNNImputer(n_neighbors=3)
knn_imputed_df = knn_imputer.fit_transform(scaled_df)

# 將補值後的資料還原為原始範圍
original_df = scaler.inverse_transform(knn_imputed_df)

In [20]:
selected_features = data.columns
original_df = pd.DataFrame(original_df, columns=selected_features)
original_df

,output,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,...,P1s,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds
0,0.0,0.0,-1.8653,3.479344,47.8891,44.044274,9.0,10.0,39.0,21.0,...,0.862046,0.116831,0.560654,0.379450,0.268039,20.706840,52.049186,91.644726,0.793068,1.208143
1,0.0,0.0,-7.2263,52.219412,137.7680,88.208099,0.0,0.0,87.0,44.0,...,0.883822,0.065994,0.572409,0.487711,0.434117,42.761478,193.812432,465.446927,0.825733,1.494237
2,0.0,0.0,-2.1004,4.411680,67.0727,55.758204,6.0,6.0,52.0,24.0,...,0.910758,0.057143,0.494202,0.319559,0.386393,29.370226,71.693573,143.387359,0.866137,1.200154
3,0.0,0.0,-0.5179,0.268220,52.6108,46.640239,6.0,6.0,43.0,20.0,...,0.908190,0.048952,0.533958,0.387559,0.470449,24.128665,49.765298,100.659835,0.862285,1.391967
4,0.0,1.0,0.5703,0.325242,37.1534,45.496309,12.0,12.0,36.0,23.0,...,0.629985,0.333300,0.474489,0.401925,0.374420,15.580870,59.559927,104.301091,0.444977,1.250833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,1.0,0.0,-0.9977,0.995405,88.4810,74.525376,11.0,11.0,66.0,34.0,...,0.790850,0.161780,0.540522,0.435705,0.420574,23.428871,92.579539,184.432320,0.686275,1.396801
936,1.0,0.0,-0.9977,0.995405,88.4810,74.525376,11.0,11.0,66.0,34.0,...,0.790850,0.161780,0.540522,0.435705,0.420574,23.428871,92.579539,184.432320,0.686275,1.396801
937,1.0,0.0,-0.9977,0.995405,88.4810,74.525376,11.0,11.0,66.0,34.0,...,0.790850,0.161780,0.540522,0.435705,0.420574,23.428871,92.579539,184.432320,0.686275,1.396801
938,1.0,0.0,-0.9977,0.995405,88.4810,74.525376,11.0,11.0,66.0,34.0,...,0.790850,0.161780,0.540522,0.435705,0.420574,23.428871,92.579539,184.432320,0.686275,1.396801


In [21]:
original_df.to_csv('knn_imputed_data.csv', index=False)